In [1]:
!python -m pip install -q pyrosetta-installer

import pyrosetta_installer
pyrosetta_installer.install_pyrosetta()

import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.stats import circstd

import rosetta
from pyrosetta import pose_from_pdb
from pyrosetta.toolbox.rcsb import pose_from_rcsb
from pyrosetta.io import poses_from_multimodel_pdb

from pyrosetta.toolbox import cleanATOM
from pyrosetta.rosetta.protocols.membrane import AddMembraneMover
from pyrosetta.rosetta.numeric import xyzVector_double_t as Vec

from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.protocols.docking import FaDockingSlideIntoContact
from pyrosetta.rosetta.protocols.minimization_packing import PackRotamersMover, MinMover

from pyrosetta.rosetta.protocols.docking import DockingPrepackProtocol ####
from pyrosetta.rosetta.core.scoring import ScoreFunctionFactory
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector, InterGroupInterfaceByVectorSelector
from pyrosetta.rosetta.core.pack.task.operation import OperateOnResidueSubset, RestrictToRepackingRLT, PreventRepackingRLT

from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover
from pyrosetta.rosetta.protocols.rigid import RigidBodySpinMover

from pyrosetta.rosetta.numeric.random import rg
from pyrosetta import init
init("-mute all")

PyRosetta install detected, doing nothing...
┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.ubuntu 2025.44+release.e5ce21e85a29101114942a7ffe084c18ead1c327 2025-10-28T16:01:33] retrieved from: http://www.pyrosetta.org


/tmp/ipykernel_237272/462082781.py:13: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  import rosetta


In [2]:
# We introduce here seeding
# Seeding allows us to get replicable results when giving an arbitrary seed number
# In this way, we can get consistent results through different executions of the same code

# Seeding (Python side)

seed = 16
random.seed(seed)
np.random.seed(seed)

# Seeding (PyRosetta side)

rg().set_seed(seed)

In [ ]:
def protein_radius_exact(p):
    coords=list()
    for r in range(1, p.total_residue()+1):
        ri=p.residue(r)
        for a in range(1, ri.natoms()+1):
            if ri.atom_is_hydrogen(a):
              continue
            v=ri.xyz(a)
            coords.append((v.x,v.y,v.z))
    m2=0.0

    for i in range(len(coords)):
        xi,yi,zi=coords[i]
        for j in range(i+1,len(coords)):
            xj,yj,zj=coords[j]
            dx,dy,dz=xi-xj,yi-yj,zi-zj
            d2=dx*dx+dy*dy+dz*dz
            if d2>m2:
              m2=d2
    return (m2**0.5)*0.5

def is_extracellular(p, chain_id):
    nx,ny,nz = n_unit.x, n_unit.y, n_unit.z
    pi = p.pdb_info()
    thr = thk
    for r in range(1, p.total_residue()+1):
        if pi.chain(r) != chain_id:
            continue
        ri = p.residue(r)
        for a in range(1, ri.natoms()+1):
            if ri.atom_is_hydrogen(a):
                continue
            xyz = ri.xyz(a)
            zproj = (xyz.x - c.x)*nx + (xyz.y - c.y)*ny + (xyz.z - c.z)*nz
            if zproj <= thr:
                return False
    return True

def merger(target, partner):
    pose0 = target.clone()
    rosetta.core.pose.append_pose_to_pose(pose0, partner)

    partner_start = target.total_residue() + 1
    for r in range(partner_start, pose0.total_residue()+1):
        pose0.pdb_info().chain(r, 'C')
    pose0.update_pose_chains_from_pdb_chains()

    return pose0

def connector(pose):
    v = rosetta.utility.vector1_int()
    v.append(1)
    rosetta.protocols.docking.setup_foldtree(pose, "A_C", v)
    dock_jump = rosetta.core.pose.get_jump_id_from_chain('C', pose)

    return dock_jump

def ensemble_maker(pose, num, phis_range, psis_range):
    for k in tqdm(range(num), desc='Ensemble making'):
        p = pose.clone()
        for i in range(5, 28):

            p.set_phi(i, p.phi(i) + random.uniform(-phis_range, phis_range))
            p.set_psi(i, p.psi(i) + random.uniform(-psis_range, psis_range))

        sfxn = ScoreFunctionFactory.create_score_function("ref2015_cart")

        fr = FastRelax(sfxn)
        fr.max_iter(200)
        fr.dualspace(True)

        fr.apply(p)
        p.dump_pdb(f"ensemble/ens_{k:03d}.pdb")

def scoring(pose, dock_jump, sfxn):
    # Configure interface analysis on interface A (receptor) vs C (ligand) using the provided scorefunction
    iam = rosetta.protocols.analysis.InterfaceAnalyzerMover(
        dock_chains='A_C',
        tracer=False, # Suppress Rosetta tracer output to keep logs clean
        sf=sfxn,
        compute_packstat=False, # Skip packstat metric
        pack_input=True, # Repack interface on the bound complex before separation to remove side-chain artifacts
        pack_separated=True, # Repack partners after separation to approximate relaxed unbound states, improving dG_interface
        use_jobname=False, # Use pose-based labels instead of jobname for consistent naming.
        detect_disulfide_in_separated_pose=False # Skip disulfide detection in the separated state
    )

    # Extract SASA-based interface metrics
    iam.set_compute_separated_sasa(True) # Compute SASA for the separated partners
    iam.set_calc_dSASA(True) # Report the delta SASA (i.e., buried surface upon binding)

    # Allow side-chain repacking before scoring (to remove clashes and standardize energies)
    iam.set_pack_input(True) # Repack in the complex
    iam.set_pack_rounds(3) #### from 3 to 5
    iam.set_pack_separated(True) # R  epack the separated partners
    iam.apply(pose)

    # Get the total complex energy with the chosen scorefunction (here, lower is better)
    tot = sfxn(pose)

    # Interface binding energy estimate (negative values indicate favorable binding)
    ddG = iam.get_separated_interface_energy()

    # Buried solvent-accessible surface area at the interface (larger often correlates with tighter interfaces)
    dsasa = iam.get_interface_delta_sasa()

    return tot, ddG, dsasa

def dock(pose, dock_jump, sfxn):
    # Get the membrane jump to keep it fixed during minimization
    jmem = pose.conformation().membrane_info().membrane_jump()

    # Bring partners into gentle contact along the docking jump
    FaDockingSlideIntoContact(dock_jump).apply(pose)

    # Define the interface between chain A (receptor) and chain C (ligand)
    selA, selB = ChainSelector('A'), ChainSelector('C')
    iface_sel = InterGroupInterfaceByVectorSelector(selA, selB)

    # Pack only side chains at the interface and freeze the complement
    tf = TaskFactory()
    tf.push_back(OperateOnResidueSubset(RestrictToRepackingRLT(), iface_sel)) # interface repack
    tf.push_back(OperateOnResidueSubset(PreventRepackingRLT(), iface_sel, True)) # complement freeze

    prm = PackRotamersMover(sfxn)
    prm.task_factory(tf)
    prm.apply(pose)

    # Local rigid minimization on docking jump
    mm = MoveMap()
    mm.set_bb(True) #### It was False
    mm.set_chi(True) #### It was False
    mm.set_jump(jmem, False) # Keep membrane jump fixed
    mm.set_jump(dock_jump, True) # Allow motion only on docking jump

    minm = MinMover()
    minm.score_function(sfxn)
    minm.movemap(mm)
    minm.min_type('lbfgs_armijo_nonmonotone')
    minm.tolerance(0.02)
    minm.apply(pose)

    return pose

def positioner(pose0, partner_name, N, dock_jump, radius):
    saved = 0
    i = 0

    sfxn = ScoreFunctionFactory.create_score_function("ref2015_memb") # try also the Franklin one

    while saved < N:
        pose = pose0.clone()

        sm = RigidBodySpinMover(dock_jump)
        sm.spin_mag(360.0)
        sm.apply(pose)

        utm = rosetta.protocols.rigid.UniformSphereTransMover(dock_jump, radius*1.5)
        utm.apply(pose)

        pose = dock(pose, dock_jump, sfxn) # Apply the docking pipeline and, then, filter using "is_extracellular"

        if is_extracellular(pose, 'C'):
            ens_id = partner_name.split('.')[0].split("_")[1]
            tot, ddG, dsasa = scoring(pose, dock_jump, sfxn)
            out_name = f"dock_{ens_id}_{i:03d}"
            rows.append((out_name, tot, ddG, dsasa))

            for k in range(pose.total_residue(), 0, -1):
                if pose.residue(k).name3() == 'MEM':
                    pose.delete_residue_slow(k)

            pose.dump_pdb(f'dock_out/{out_name}.pdb')

            saved += 1
            i += 1
            pbar.update(1)

In [4]:
# Load and process the VIP
# We can use the 2RRH PDB, since it is more stable

vip_id = '2RRH'
medoid_id = 0

pose_from_rcsb(vip_id)
poses = [pose for pose in poses_from_multimodel_pdb(f"{vip_id}.pdb")]

poses[medoid_id].dump_pdb(f"{vip_id}.medoid.pdb")
cleanATOM(f"{vip_id}.medoid.pdb")
vip_pose = pose_from_pdb(f"{vip_id}.medoid.clean.pdb")

# We are adding here a new processing line for the VIP
# Because we want to process the VIP in order to delete the last residue (i.e., the 29th one)
vip_pose.delete_residue_range_slow(29, 29) # The argument must express a range (in this case "from 29 to 29")


# Load and process the CRTH2

crth2_id = "8XXU" # You must load this manually in your environment!

raw_input = f"{crth2_id}.pdb"
cleanATOM(f"{raw_input}")
final_input = f"{crth2_id}.clean.pdb"
crth2_pose = pose_from_pdb(final_input)

subposes = crth2_pose.split_by_chain()
for subpose in subposes:
    if subpose.pdb_info().chain(1) == 'A':
        crth2_pose = subpose
        break

addmem = AddMembraneMover("from_structure")
addmem.apply(crth2_pose)
os.rename("out.span", f"{crth2_id}.span")

In [5]:
# Retrieve the membrane information and get the radius of the receptor

confo = crth2_pose.conformation()
mi = confo.membrane_info()

c = mi.membrane_center(confo)
n = mi.membrane_normal(confo)
nl = (n.x**2 + n.y**2 + n.z**2) ** 0.5
n_unit = Vec(n.x/nl, n.y/nl, n.z/nl)

thk = mi.membrane_thickness()

r_crth2 = protein_radius_exact(crth2_pose)

print(f'Maximum radius of the receptor: {round(r_crth2, 2)} Å')

# Calculate the (circular) standard deviation of the angles of the VIP

phis = list()
psis = list()

for pose in poses:
    for i in range(5, 28):
        phis.append(pose.phi(i))
        psis.append(pose.psi(i))

phi_std = np.rad2deg(circstd(np.deg2rad(phis), low=-np.pi, high=np.pi, normalize=False))
psi_std = np.rad2deg(circstd(np.deg2rad(psis), low=-np.pi, high=np.pi, normalize=False))

print(f'VIP multimodel phi standard deviation: {round(phi_std, 2)}°')
print(f'VIP multimodel psi standard deviation: {round(psi_std, 2)}°')

Maximum radius of the receptor: 35.83 Å
VIP multimodel phi standard deviation: 5.19°
VIP multimodel psi standard deviation: 6.64°


In [6]:
# This code uses the functions defined above to generate an ensemble for the VIP and to use it

# Generate the ensemble

# 5 VIPs for 20 positions means a total output of 100 docking files (5 x 20)

num_vips = 10
num_positions = 100

In [7]:
os.makedirs('ensemble', exist_ok=True)

ensemble_maker(vip_pose, num=num_vips, phis_range=phi_std, psis_range=psi_std) # This function make the ensemble using the angles' ranges

Ensemble making: 100%|██████████| 10/10 [00:41<00:00,  4.13s/it]


In [ ]:
# Perform the docking

os.makedirs('dock_out', exist_ok=True)

pbar = tqdm(total=num_vips*num_positions, desc='Positions making') # This is just for the advancement bar...

# This list will collect the metrics for each docking file we make
# It is enriched inside the positioner function

rows = list()

for vip_ens in os.listdir('ensemble'): # This cycle allows us to use a combinatorial approach
    vip_pose = pose_from_pdb(f"ensemble/{vip_ens}") # Load the poses of the ensemble

    pose0 = merger(crth2_pose, vip_pose) # Merge the poses to make a VIP-CRTH2 unique pose
    dock_jump = connector(pose0) # Extract the docking jump index

    sfxn = ScoreFunctionFactory.create_score_function("ref2015_memb") # try also the Franklin one

    ####
    ppk = DockingPrepackProtocol()
    ppk.set_partners("A_C")
    ppk.set_scorefxn_pack(sfxn)
    ppk.set_rt_min(True)
    ppk.set_sc_min(True)
    ppk.apply(pose0)
    ####

    positioner(pose0, vip_ens, num_positions, dock_jump, r_crth2) # Perform the docking

pbar.close()

Positions making:  10%|█         | 101/1000 [2:06:00<25:27:14, 101.93s/it]

In [ ]:
# Here we use the "rows" list to make a pandas dataframe containing the metrics results of our docking

# Make a dictionary that will give the name to our columns

data = list()

for path, score, ddg, dsasa in rows:
    data.append({
        'model_path': path,
        'score': float(score),
        'dG_interface': float(ddg),
        'dSASA_int': float(dsasa),
    })

# Make the actual dataframe

df = pd.DataFrame(data, columns=['model_path','score','dG_interface','dSASA_int'])

# Sort and save

df = df.sort_values(['score'], kind='mergesort').reset_index(drop=True)
df.to_csv('summary.csv', index=False)

In [ ]:
# — imports
import pandas as pd
import numpy as np
from tqdm import tqdm

df_filtered = df.copy()

# — hard filters
m = df_filtered["dSASA_int"] > 0
df_filtered = df_filtered[m]

m = df_filtered["dG_interface"] < 0
df_filtered = df_filtered[m]

m = df_filtered["dSASA_int"] >= 600
df_filtered = df_filtered[m]

df_filtered["bind_density"] = 100 * df_filtered["dG_interface"] / df_filtered["dSASA_int"]

q25 = df_filtered["bind_density"].quantile(0.25)
df_filtered = df_filtered[df_filtered["bind_density"] <= q25]

# — export
df_filtered = df_filtered.sort_values(["bind_density","dG_interface"]).reset_index(drop=True)

In [ ]:
df_filtered.head(100)

,model_path,score,dG_interface,dSASA_int,bind_density
0,dock_002_428,-218.590196,-16.087124,654.265945,-2.458805
1,dock_001_520,-154.572100,-6.404239,627.364712,-1.020816
2,dock_009_327,-283.387368,-6.999148,703.390300,-0.995059
3,dock_003_032,-304.706039,-5.316310,620.427098,-0.856879
4,dock_004_617,-229.107528,-5.361377,639.125633,-0.838861
5,dock_005_661,1476.257235,-5.386820,646.882132,-0.832736
6,dock_008_261,207.414370,-6.195951,798.703716,-0.775751
7,dock_005_921,-138.802174,-5.953114,778.332520,-0.764855
8,dock_000_783,-126.490610,-5.124755,685.486496,-0.747608
9,dock_002_175,-245.706731,-4.594440,627.855150,-0.731767


In [ ]:
# https://chatgpt.com/c/6911afa2-d598-8330-bd73-68565398bcb7
# https://chatgpt.com/c/6915abc0-1ce8-832e-b56a-f3c02b7390be

In [ ]:
for name in tqdm(df_filtered['model_path'].iloc[:5]):
    pose = pose_from_pdb(f"dock_out/{name}.pdb")
    pose.dump_pdb(f"best/{name}.pdb")

100%|██████████| 5/5 [00:01<00:00,  3.05it/s]
